In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

READ NPZ FILE

In [2]:
data_seq = np.load("sensor_data_all_days.npz")
data = data_seq['data']
t = data_seq['timestamp']

In [3]:
data.shape

(86400, 7, 320)

In [4]:
t.shape

(86400,)

In [5]:
t[-1]

'2024-12-08 03:59:55.000000'

RESAMPLE DATA FROM 64 HZ TO 30 HZ

In [2]:
import h5py

In [3]:
def save_h5(filepath, dataset_name, data):
        """
        Save data to an .h5 file.

        Parameters:
        - filepath (str): Path to the .h5 file.
        - dataset_name (str): Name of the dataset to save.
        - data (numpy.ndarray): Data to save.
        """
        with h5py.File(filepath, 'w') as h5file:
            h5file.create_dataset('data', data=data, compression='gzip', compression_opts=9)
        print(f"Data saved to {filepath} under dataset '{dataset_name}'")

In [ ]:
# # Read the datasets and combine them
# def combine_channels(h5_filepath):
#     with h5py.File(h5_filepath, 'r') as h5file:
#         # List all channel datasets
#         datasets = sorted(h5file.keys())  # Ensure channels are loaded in order
#         num_channels = len(datasets)

#         # Read the first channel to get the shape
#         first_channel = h5file[datasets[0]]
#         new_num_seq, new_samples = first_channel.shape

#         # Initialize an empty array to hold the combined data
#         combined_data = np.empty((new_num_seq, num_channels, new_samples), dtype=first_channel.dtype)

#         # Load each channel and stack into the combined array
#         for i, dataset_name in enumerate(datasets):
#             print(f"Loading {dataset_name}...")
#             combined_data[:, i, :] = h5file[dataset_name][:]
    
#     return combined_data

In [4]:
with h5py.File('sensor_data_seq.h5', 'r') as h5file:
    # List all channel datasets
    X64 = h5file['data'][:]

In [5]:
# Combine channels
# X64 = combine_channels(data_64hz_path)
print("Data shape:", X64.shape)

Data shape: (86400, 7, 320)


In [6]:
# Define the original and target sample sizes
original_length = 320
target_length = 150

# Compute fractional indices for the target length
fractional_indices = np.linspace(0, original_length - 1, target_length)

# Round the indices to the nearest integers
indices_to_keep = np.round(fractional_indices).astype(int)
indices_to_keep

array([  0,   2,   4,   6,   9,  11,  13,  15,  17,  19,  21,  24,  26,
        28,  30,  32,  34,  36,  39,  41,  43,  45,  47,  49,  51,  54,
        56,  58,  60,  62,  64,  66,  69,  71,  73,  75,  77,  79,  81,
        83,  86,  88,  90,  92,  94,  96,  98, 101, 103, 105, 107, 109,
       111, 113, 116, 118, 120, 122, 124, 126, 128, 131, 133, 135, 137,
       139, 141, 143, 146, 148, 150, 152, 154, 156, 158, 161, 163, 165,
       167, 169, 171, 173, 176, 178, 180, 182, 184, 186, 188, 191, 193,
       195, 197, 199, 201, 203, 206, 208, 210, 212, 214, 216, 218, 221,
       223, 225, 227, 229, 231, 233, 236, 238, 240, 242, 244, 246, 248,
       250, 253, 255, 257, 259, 261, 263, 265, 268, 270, 272, 274, 276,
       278, 280, 283, 285, 287, 289, 291, 293, 295, 298, 300, 302, 304,
       306, 308, 310, 313, 315, 317, 319])

In [7]:
# Subsample the data using these indices
X30 = X64[:, :, indices_to_keep]

print("Original shape:", X64.shape)
print("Downsampled shape:", X30.shape)

Original shape: (86400, 7, 320)
Downsampled shape: (86400, 7, 150)


In [8]:
X30[0]

array([[ 0.72851363, -1.97226325,  1.44096863, ..., -0.30792527,
        -1.95175043,  1.48047396],
       [-1.34932879,  0.1627911 ,  1.5037489 , ...,  1.5708463 ,
        -0.2349665 ,  0.54133935],
       [ 1.943272  , -1.93096235,  1.86852888, ..., -1.00515273,
         1.49717586, -1.05613862],
       ...,
       [48.54540388, 13.11833533, 39.50169208, ..., 45.18424674,
        43.07128692, 42.95263446],
       [34.74493881, 36.53193387, 34.35235189, ..., 29.18680941,
        33.59140713, 34.53353279],
       [ 0.        ,  0.        ,  0.        , ...,  1.        ,
         1.        ,  1.        ]])

In [9]:
data_30hz_path = r'X30.h5'

save_h5(data_30hz_path, 'data', X30)

Data saved to X30.h5 under dataset 'data'


In [ ]:
from scipy.signal import resample

In [7]:
# Parameters
original_frequency = 64  # Original frequency (in Hz)
new_frequency = 30  # New frequency (in Hz)
sequence_length = 320  # 320 samples for 5 seconds at original frequency

In [8]:
# Calculate the new number of samples per sequence
new_sequence_length = int(sequence_length * new_frequency / original_frequency)

# Resample each sequence
resampled_sequences = np.array([
    resample(sequence.T, new_sequence_length).T  # Resample each sequence (7, 320) -> (7, new_sequence_length)
    for sequence in data
])

data = resampled_sequences

# Display the shape of the resampled data
print("Resampled data shape:", data.shape)

Resampled data shape: (86400, 6, 150)


In [9]:
np.savez_compressed('X_30.npz', timestamp=t, data=data)

GET THE ACCELEROMETER DATA FROM THE SEQUENCES

In [11]:
acc_data = data[:, 0:3, :]

In [12]:
acc_data[:5,:,:]

array([[[ 0.53318214,  0.42503497, -1.18262025, ..., -0.30819584,
         -1.35185685,  0.25438791],
        [ 0.27778524,  1.2567066 ,  0.18742472, ...,  1.29662915,
         -0.36952494, -0.11866601],
        [-0.3131505 , -0.29049202, -0.97024155, ...,  1.56263133,
         -1.4313148 ,  0.45225118]],

       [[ 0.98310011,  0.46642888, -1.07128165, ..., -0.70044903,
          0.15718222,  1.34113926],
        [-0.04535848,  1.27734019,  0.4928513 , ...,  0.34049488,
         -0.01420413, -0.70921527],
        [ 0.94272288,  0.93957894, -0.48455656, ...,  0.13058852,
          0.82484492, -0.23871198]],

       [[ 0.55293449, -1.13348641, -1.27410082, ...,  1.2548377 ,
         -0.4526727 ,  0.3072528 ],
        [ 0.22472251, -0.96915385, -0.8678195 , ...,  0.25472418,
         -0.57843839,  0.73577172],
        [-0.07593484,  1.1181429 ,  0.91800281, ...,  1.06306661,
         -0.5799588 , -0.32849995]],

       [[ 0.31268351,  0.75244547,  0.5238766 , ...,  0.68327422,
          

In [13]:
np.savez_compressed('X_30_acc.npz', timestamp=t, acc_data=acc_data)